In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import warnings
warnings.filterwarnings("ignore")

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import gc
import seaborn as sns
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data = pd.read_csv('/kaggle/input/santander-customer-transaction-prediction/train.csv')
data.info()

In [ ]:
data.head()

In [ ]:
data.drop('ID_code', 1, inplace=True)
data.describe()

In [ ]:
missing = False
for i in data.columns:
    if len(pd.notnull(data[i])==True)!=len(data):
        print (i, ": ",len(pd.notnull(data[i])))
        missing = True
if not missing:
    print("No missing Data in train data.")

In [ ]:
test = pd.read_csv('../input/santander-customer-transaction-prediction/test.csv')
test.head()

In [ ]:
test.info()

In [ ]:
missing = False
for i in test.columns:
    if len(pd.notnull(test[i])==True)!=len(test):
        print (i, ": ",len(pd.notnull(test[i])))
        missing = True
if not missing:
    print("No missing Data in test.")

### EDA

In [ ]:
sns.countplot('target', data=data)

In [ ]:
str( np.round(len(data[data['target']==1]) /(len(data[data['target']==0]) + len(data[data['target']==0]))*100,2)) +' %, ratio of positive class' 

In [ ]:
plt.figure(figsize=(15,15))

sns.heatmap(data.corr(), cmap='coolwarm')

Totally uncorellated Features

In [ ]:
init_cols = data.columns[1:]

In [ ]:
stat_table = pd.DataFrame()
means_0 = []
means_1 = []
for i in init_cols:
    means_0.append(data[data['target']==0][i].mean())
    means_1.append(data[data['target']==1][i].mean())

stat_table["feat"] = init_cols
stat_table["0"] = means_0
stat_table["1"] = means_1

stat_table["Class 0"] = stat_table.apply(lambda x: x["0"]/(x["0"]+x["1"]), 1)
stat_table["Class 1"] = stat_table.apply(lambda x: x["1"]/(x["0"]+x["1"]), 1)
stat_table = stat_table[["feat", "Class 0", "Class 1"]]

plt.figure(figsize=(15,15))
sns.heatmap(stat_table[["Class 0", "Class 1"]], cmap='coolwarm')



Spotted several features where their class means differ from each other

In [ ]:
stat_table["var"] = stat_table.apply(lambda x: np.abs(x["Class 0"] - x["Class 1"])>0.05  , 1)
stat_table = stat_table[stat_table['var']==True]
# sns.heatmap(stat_table[["Class 0", "Class 1"]], cmap='coolwarm')
#stat_table

In [ ]:
n_rows = 5
fig , axs = plt.subplots (n_rows, len(stat_table)//n_rows, figsize=(25,15))
cols = stat_table['feat'].values
for i,ax in enumerate(axs.flatten()):
    
    sns.distplot(data[data['target']==0][cols[i]] , ax=ax)
    sns.distplot(data[data['target']==1][cols[i]] , ax=ax)
    
plt.tight_layout()
plt.show()

In [ ]:
data.head()

In [ ]:
from sklearn import mixture

n_rows = 5
n_components = 2 
fig , axs = plt.subplots (n_rows, len(stat_table)//n_rows, figsize=(25,15))
cols = stat_table['feat'].values
all_data = pd.concat([data, test])
for i,ax in enumerate(axs.flatten()):
    
    x = all_data[cols[i]].values

    
    clf = mixture.GaussianMixture(n_components=n_components, )
    clf.fit(x.reshape(-1,1))
    samples = clf.sample(len(x))
    samples_cp1 = samples[0][samples[1] ==0]
    samples_cp2 = samples[0][samples[1] ==1]
    sns.distplot(samples_cp1, kde=False, ax=ax)
    sns.distplot(samples_cp2, kde=False, ax=ax)
    
    all_data[cols[i]+'prob'] = clf.predict_proba(x.reshape(-1,1))[:,1]
        
plt.tight_layout()
plt.show()

In [ ]:
n_rows = 30

fig , axs = plt.subplots (n_rows, len(init_cols)//n_rows, figsize=(25,80))

for i,ax in enumerate(axs.flatten()):
    tmp = data[[init_cols[i], 'target']].copy()
    x = tmp[[init_cols[i]]].values
    bins = np.histogram(x)[1]
    bins_indexes =  np.digitize(x, bins)
    tmp['index'] = bins_indexes
    
    tmp = tmp.groupby('index')['target'].mean()
    tmp.plot(ax=ax)
    ax.set_title(init_cols[i])
        
plt.tight_layout()
plt.show()

In [ ]:
data = all_data.iloc[:len(data),:].drop('ID_code', 1)
test = all_data.iloc[len(data):,:].drop('target', 1)

In [ ]:
data['average_row'] = data[init_cols] .apply(lambda x: np.round(np.mean(x),5), 1)
test['average_row'] = test[init_cols] .apply(lambda x: np.round(np.mean(x),5), 1)

In [ ]:
data['std_row'] = data[init_cols] .apply(lambda x: np.round(np.std(x),5), 1)
test['std_row'] = test[init_cols] .apply(lambda x: np.round(np.std(x),5), 1)

In [ ]:
fig , axs = plt.subplots (1, 2, figsize=(15,5)) 

sns.distplot(data['average_row'] , ax=axs[0], label='Train Data' )
sns.distplot(test['average_row'] , ax=axs[0], label='Tets Data')
axs[0].legend()
sns.distplot(data[data['target']==0]['average_row'] , ax=axs[1], label='Class 0' )
sns.distplot(data[data['target']==1]['average_row'] , ax=axs[1], label='Class 1')
axs[1].legend()
plt.tight_layout()
plt.show()

In [ ]:
fig , axs = plt.subplots (1, 2, figsize=(15,5)) 

sns.distplot(data['std_row'] , ax=axs[0], label='Train Data' )
sns.distplot(test['std_row'] , ax=axs[0], label='Tets Data')
axs[0].legend()
sns.distplot(data[data['target']==0]['std_row'] , ax=axs[1], label='Class 0' )
sns.distplot(data[data['target']==1]['std_row'] , ax=axs[1], label='Class 1')
axs[1].legend()
plt.tight_layout()
plt.show()

Train and test dets are balanced. Difference in mean between the two classes

In [ ]:
fig , axs = plt.subplots (1, 2, figsize=(15,5)) 

sns.distplot(np.mean(data[init_cols]) , ax=axs[0], label='Train Data' )
sns.distplot(np.mean(test[init_cols]) , ax=axs[0], label='Tets Data')
axs[0].legend()
sns.distplot(np.mean(data[data['target']==0][init_cols]) , ax=axs[1], label='Class 0' )
sns.distplot(np.mean(data[data['target']==1][init_cols]) , ax=axs[1], label='Class 1')
axs[1].legend()
plt.tight_layout()
plt.show()

from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler


X = data[cols].values
y = data['target'].values


scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

pca = PCA(2).fit(X)
X = pca.transform(X)

df = pd.DataFrame()

df['target'] = y
df[['x', 'y']] = X
sns.scatterplot('x', 'y', 'target', data=df)

print(pca.explained_variance_ratio_)

No Patterns in data

In [ ]:
plt.figure(figsize=(35,8))

n_unique_data = {i:len(np.unique(data[i])) for i in init_cols}
n_unique_data = {k: v for k, v in sorted(n_unique_data.items(), key=lambda item: item[1])}
sns.barplot(list(n_unique_data.keys()), list(n_unique_data.values()))
plt.xticks(rotation=90)
plt.show()


In [ ]:
plt.figure(figsize=(35,8))

n_unique_test = {i:len(np.unique(test[i])) for i in init_cols}
n_unique_test = {k: v for k, v in sorted(n_unique_test.items(), key=lambda item: item[1])}
sns.barplot(list(n_unique_test.keys()), list(n_unique_test.values()))
plt.xticks(rotation=90)
plt.show()


In [ ]:
all_data = pd.concat([data, test])

for i in init_cols:
    counts = all_data[i].value_counts()
    all_data[i+"_count"] =  all_data[i].map(counts)

In [ ]:
powers = [2, 3]
for p in powers:
    for i in init_cols:
        all_data[i+"_"+str(p)] = np.power(all_data[i], p)

In [ ]:
data = all_data.iloc[:len(data),:].drop('ID_code', 1)
test = all_data.iloc[len(data):,:].drop('target', 1)

In [ ]:
import gc
del all_data
gc.collect()

In [ ]:
from sklearn.manifold import TSNE
from sklearn.preprocessing import MinMaxScaler


X = subsampled_data[cols].values
y = subsampled_data['target'].values


scaler = MinMaxScaler()
scaler.fit(X)
X = scaler.transform(X)

X = TSNE(2, n_jobs=-1).fit_transform(X)

df = pd.DataFrame()

df['target'] = y
df[['x', 'y']] = X
sns.scatterplot('x', 'y', 'target', data=df)



## Base Model 

In [ ]:
submission = test.copy()
submission['target'] = 1
submission = submission [['ID_code', 'target']]
# submission.to_csv('submission.csv', index=False)

Always prediticting 1 leads to a score of 50%, it means that the test set is balanced.

In [ ]:
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, roc_auc_score
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV

In [ ]:
from imblearn.over_sampling import RandomOverSampler

# cols_train = stat_table['feat']
X = data.drop('target', 1).values
y = data['target'].values
print ('Original Dataset', Counter(y))

scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)


nm = RandomOverSampler()
X, y = nm.fit_resample(X, y)
print('Resampled dataset shape %s' % Counter(y))


grid={"C":np.logspace(-3,3,4)}
     #"class_weight": ['balanced', None]}# l1 lasso l2 ridge
logreg=LogisticRegression()
logreg_cv=GridSearchCV(logreg,grid,cv=10)
logreg_cv.fit(X,y)

print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)


scores_f1 = []
scores_roc = []

C = logreg_cv.best_params_['C']
#class_weight = logreg_cv.best_params_['class_weight']

kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

for train_ix, test_ix in kfold.split(X, y):
    train_X, test_X = X[train_ix], X[test_ix]
    train_y, test_y = y[train_ix], y[test_ix]
    
    clf = LogisticRegression(C=C, )
    clf.fit(train_X, train_y)
    y_hat = clf.predict(train_X)
    scores_f1.append(f1_score(train_y, y_hat))
    
    y_hat = clf.predict_proba(train_X)[:,1]
    scores_roc.append(roc_auc_score(train_y, y_hat))

print ('F1 Score: %f, +/-%f'%(np.mean(scores_f1), np.std(scores_f1)))
print ('ROC AUC Score: %f, +/-%f'%(np.mean(scores_roc), np.std(scores_roc)))

clf = LogisticRegression(C=C)
clf.fit(X, y)
    
coefs = np.abs(clf.coef_[0])
indices = np.argsort(coefs)[::-1]

plt.figure()
plt.title("Feature importances (Logistic Regression)")
plt.bar(range(15), coefs[indices[:15]],
       color="r", align="center")
plt.xticks(range(15), data.columns[indices[:15]], rotation=45, ha='right')
plt.subplots_adjust(bottom=0.3)

In [ ]:
from sklearn.naive_bayes import GaussianNB


# cols_train = stat_table['feat']
X = data.drop('target', 1).values
y = data['target'].values
print ('Original Dataset', Counter(y))

nm = RandomOverSampler()
X, y = nm.fit_resample(X, y)

scores_f1 = []
scores_roc = []


kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

for train_ix, test_ix in kfold.split(X, y):
    train_X, test_X = X[train_ix], X[test_ix]
    train_y, test_y = y[train_ix], y[test_ix]
    
    clf = GaussianNB()
    clf.fit(train_X, train_y)
    
    y_hat = clf.predict(train_X)
    scores_f1.append(f1_score(train_y, y_hat))
    
    y_hat = clf.predict_proba(train_X)[:,1]
    scores_roc.append(roc_auc_score(train_y, y_hat))

print ('F1 Score: %f, +/-%f'%(np.mean(scores_f1), np.std(scores_f1)))
print ('ROC AUC Score: %f, +/-%f'%(np.mean(scores_roc), np.std(scores_roc)))

clf = GaussianNB()
clf.fit(X, y)
y_hat = clf.predict_proba(X)[:,1]
print ('ROC AUC Score: %f'%(roc_auc_score(y, y_hat)))


In [ ]:
import lightgbm as lgb

gbdt_param = {
    # Core Parameters
    'objective': 'binary',
    'boosting': 'gbdt',
    'learning_rate': 0.01,
    'num_leaves': 15,
    'tree_learner': 'serial',
    'num_threads': 8,
    'seed': 42,
    
    # Learning Control Parameters
    'max_depth': -1,
    'min_data_in_leaf': 50,
    'min_sum_hessian_in_leaf': 10,  
    'bagging_fraction': 0.6,
    'bagging_freq': 5,
    'feature_fraction': 0.05,
    'lambda_l1': 1.,
    'bagging_seed': 42,
    
    # Others
    'verbosity ': 1,
    'boost_from_average': False,
    'metric': 'auc',
    #'device' : 'gpu'
}


# cols_train = stat_table['feat']
X = data.drop('target', 1).values
y = data['target'].values
print ('Original Dataset', Counter(y))
nm = RandomOverSampler()
X, y = nm.fit_resample(X, y)

scores_f1 = []
scores_roc = []


kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

for train_ix, test_ix in kfold.split(X, y):
    train_X, test_X = X[train_ix], X[test_ix]
    train_y, test_y = y[train_ix], y[test_ix]
    evals_result = {}
    trn_data = lgb.Dataset(train_X, label=train_y)
    val_data = lgb.Dataset(test_X, label=test_y)
    
    lgb_clf = lgb.train(gbdt_param, trn_data, 10000,
             valid_sets=[trn_data, val_data], early_stopping_rounds=5000,
                        verbose_eval=1000, evals_result=evals_result)


{'bootstrap': True, 'ccp_alpha': 0.0, 'class_weight': None, 'criterion': 'gini', 'max_depth': None, 'max_features': 0.32888326498603004, 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 0.01363936719149417, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 80, 'n_jobs': None, 'oob_score': False, 'random_state': None, 'verbose': 0, 'warm_start': False}

## Submissions

In [ ]:
import lightgbm as lgb

gbdt_param = {
    # Core Parameters
    'objective': 'binary',
    'boosting': 'gbdt',
    'learning_rate': 0.01,
    'num_leaves': 15,
    'tree_learner': 'serial',
    'num_threads': 8,
    'seed': 42,
    
    # Learning Control Parameters
    'max_depth': -1,
    'min_data_in_leaf': 50,
    'min_sum_hessian_in_leaf': 10,  
    'bagging_fraction': 0.6,
    'bagging_freq': 5,
    'feature_fraction': 0.05,
    'lambda_l1': 1.,
    'bagging_seed': 42,
    
    # Others
    'verbosity ': 1,
    'boost_from_average': False,
    'metric': 'auc',
#    'device' : 'gpu'
}



# cols_train = stat_table['feat']
X = data.drop('target', 1).values
y = data['target'].values
nm = RandomOverSampler()
X, y = nm.fit_resample(X, y)

trn_data = lgb.Dataset(X, label=y)

lgb_clf = lgb.train(gbdt_param, trn_data, 100000)


In [ ]:
submission = test.copy()
submission['target'] = lgb_clf.predict(submission.drop('ID_code', 1).values)
submission = submission [['ID_code', 'target']]

submission.to_csv('submission.csv', index=False)

In [ ]:
submission.info()

In [ ]:
sns.distplot(submission['target'])